# Metaphor Detection via In Context Learning with Local Model in Ollama

In this notebook, I will show the workflow of metaphor detection via in context learning(ICL) with local model. Here, we host local model in Ollama.

Let's start with the dependencies. To run this notebook, you need the following packages: ollama, pandas.

Run the following script to install these packages.

In [ ]:
!pip install ollama pandas

Import packages.

In [ ]:
import pandas as pd
from ollama import chat
from ollama import ChatResponse
import copy

Next, you'll need to load the prompt(s). The prompt(s) are stored as csv. You can use the following script to load it.

In [ ]:
prompt_fp="prompt/prompt_cot.csv"
prompt_df=pd.read_csv(prompt_fp,index_col=0)
pids=list(set(prompt_df.index.to_list()))
p_strats={}
p_strats_info={}
for pid in pids:
    chat_temp_df=prompt_df.loc[pid]
    chat_temp=[]
    for i in range(0,chat_temp_df.shape[0]):
        item=chat_temp_df.iloc[i]
        chat_temp.append({"role":item["role"],"content":item["content"]})
    p_strats[pid]=chat_temp
    p_strats_info[pid]=chat_temp_df.iloc[0]["name"]

The prompts are stored in the dictionary p_strats. You may access prompt via prompt_id(pid). For a full list of pid:prompt strategy, simply check:

In [ ]:
p_strats_info

{100: 'V2a. Chain-of-thought, 4 shots, Original conventional vs creative ratio',
 101: 'V2b. Chain-of-thought, 4 shots, Even conventional vs creative ratio',
 102: 'V2c. Chain-of-thought, 8 shots, Original conventional vs creative ratio',
 103: 'V2d. Chain-of-thought, 8 shots, Even conventional vs creative ratio',
 104: 'V6a. Chain-of-thought, 4 shots, Original conventional vs creative ratio',
 105: 'V6b. Chain-of-thought, 4 shots, Even conventional vs creative ratio',
 106: 'V6c. Chain-of-thought, 8 shots, Original conventional vs creative ratio',
 107: 'V6d. Chain-of-thought, 8 shots, Even conventional vs creative ratio',
 108: 'V7a. Chain-of-thought, 4 shots, Original conventional vs creative ratio',
 109: 'V7b. Chain-of-thought, 4 shots, Even conventional vs creative ratio',
 110: 'V7c. Chain-of-thought, 8 shots, Original conventional vs creative ratio',
 111: 'V7d. Chain-of-thought, 8 shots, Even conventional vs creative ratio'}

Select your prompt strategy via prompt id using the folllowing script:

In [ ]:
p_strat=p_strats[pid]

After the prompts are loaded, you'll need to load the test text. To load it from our corpus, simply do:

In [ ]:
ds_fp="data/metaphor_dataset.csv"
ds_df=pd.read_csv(ds_fp,index_col=0)

You may input your own test text in the following script. Or, alternatively, you may choose a sample test text from our corpus, for example:

In [ ]:
test_text=ds_df.iloc[0]["plain"]
print(test_text)

 " something is fishy in the state of universal . " 

about ten years back , with the unexpected success of mad max and the road warrior , post-apocalypse nitty-gritty survival yarns became popular at the movies . 

we've always had movies of this nature ; on the beach , the end of the world , damnation alley , the ultimate warrior , and so on . 

to date , the most smoothly done were straightforward " haircuts " of the classic western plot , like the lone gunman who comes to town and protects the widow and the son against an evil organization , usually one in possession of some critical resource , like water , feed range , or a mining claim . 

most of these grew out of venerable , but solid hero yarns like the virginian and shane . 

 ( my personal favorite is a patrick swayze movie called steel dawn , which was fairly well made on a small budget . ) 

now we have waterworld , which again brings the traditional lone gunman to town to rescue the young widow and her daughter . 

 ( wel

Next, as the last step required before run, you need to specify a model.

The models we used in our paper are:

llama3.2:1b

llama3.2:3b

llama3.1:8b

deepseek-r1:8b


In [ ]:
modelid="llama3.2:1b"

Note: to use the model you specify, you'll need ollama installed and started. You may download Ollama here:

https://ollama.com/

And if ollama is not started, simply run:

In [ ]:
!ollama start

Also, if you haven't download the model you specified, you may use the following script to download the model.

(Here I use llama3.2:1b as an example.)

In [ ]:
!ollama pull llama3.2:1b

Pack the chat of the sample text under prompt strategy, send chat to model for inferring, and retrieve result.

In [ ]:
p_strat=p_strats[pid]
ct=copy.deepcopy(p_strat)
ct[-1]["content"]=ct[-1]["content"].replace("[#TEST_TEXT]",test_text)
cr=chat(model=modelid, messages=ct)
rs=cr.message.content

View the result.

In [ ]:
print(rs)